In [1]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
import numpy as np
import pandas as pd
import os
from PIL import Image
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
#import seaborn as sns
import glob
%matplotlib inline 
from sklearn.model_selection import train_test_split

Using Theano backend.


In [2]:
def convert_image_to_data(image, WIDTH, HEIGHT):
    image_resized = Image.open(image).resize((WIDTH, HEIGHT))
    image_array = np.array(image_resized)
    return image_array

In [3]:
dog_files=glob.glob("/home/aries/Desktop/train/dog*")
#dog_files=dog_files[:2000]
cat_files = glob.glob("/home/aries/Desktop/train/cat*")
#cat_files=cat_files[:2000]

In [4]:
width=224
height=224
epoch=100
batch_size=128

In [5]:
cat_files

['/home/aries/Desktop/train/cat.651.jpg',
 '/home/aries/Desktop/train/cat.1885.jpg',
 '/home/aries/Desktop/train/cat.10034.jpg',
 '/home/aries/Desktop/train/cat.3588.jpg',
 '/home/aries/Desktop/train/cat.6907.jpg',
 '/home/aries/Desktop/train/cat.11649.jpg',
 '/home/aries/Desktop/train/cat.7661.jpg',
 '/home/aries/Desktop/train/cat.9757.jpg',
 '/home/aries/Desktop/train/cat.5441.jpg',
 '/home/aries/Desktop/train/cat.10909.jpg',
 '/home/aries/Desktop/train/cat.11463.jpg',
 '/home/aries/Desktop/train/cat.9567.jpg',
 '/home/aries/Desktop/train/cat.9723.jpg',
 '/home/aries/Desktop/train/cat.3479.jpg',
 '/home/aries/Desktop/train/cat.8215.jpg',
 '/home/aries/Desktop/train/cat.9056.jpg',
 '/home/aries/Desktop/train/cat.7629.jpg',
 '/home/aries/Desktop/train/cat.4236.jpg',
 '/home/aries/Desktop/train/cat.844.jpg',
 '/home/aries/Desktop/train/cat.6388.jpg',
 '/home/aries/Desktop/train/cat.5430.jpg',
 '/home/aries/Desktop/train/cat.9551.jpg',
 '/home/aries/Desktop/train/cat.12385.jpg',
 '/home/

In [6]:
def creating_test_train_datset(WIDTH,HEIGHT):
    #cat=glob.glob("/home/ashiya/Downloads/dog_vs_cat/train//cat.*")
    #dog=glob.glob("/home/ashiya/Downloads/dog_vs_cat/train/dog*")
    
    cat_list=[convert_image_to_data(i, WIDTH, HEIGHT) for i in cat_files]
    dog_list=[convert_image_to_data(i,WIDTH,HEIGHT) for i in dog_files]
    
    y_cat_list=np.zeros(len(cat_list))
    y_dog_list=np.ones(len(dog_list))
    x=np.concatenate([cat_list,dog_list])
    y=np.concatenate([y_cat_list,y_dog_list])
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1, stratify=y)
    return X_train, X_test, y_train, y_test

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [8]:
#divide in categories by giving itn numbers like 0,1,2
num_classes = 2
X_train,X_test,y_train,y_test=creating_test_train_datset(224,224)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print X_train.shape[1:]

(224, 224, 3)


In [9]:
#include top false karne se hati hai fully connected layer
model=keras.applications.vgg16.VGG16(include_top=False, weights='imagenet',input_shape=(height,width,3))

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [11]:
for layer in model.layers[:3]:
    layer.trainable = False



#Adding custom Layers 
x = model.output
x = Flatten()(x)

x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(510, activation="relu")(x)
predictions = Dense(2, activation="softmax")(x)

In [12]:
model = Model(inputs=model.input, outputs=predictions)
#opt = keras.optimizers.adam(lr=0.0001,beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.)
#model.compile(loss='binary_crossentropy', optimizer=opt,metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [14]:
epochs=10
batch_size=128

In [15]:
#Adding custom Layers 
#x = model.output
#x = Flatten()(x)
#x = Dense(1024, activation="relu")(x)
#x = Dropout(0.5)(x)
#x = Dense(1024, activation="relu")(x)
#predictions = Dense(16, activation="softmax")(x)
# creating the final model 
opt = keras.optimizers.adam(lr=0.0001,beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.)
#rmsprop
#    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255


In [ ]:
model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test))

Train on 3200 samples, validate on 800 samples
Epoch 1/10


In [ ]:
test_files=glob.glob("/home/ashiya/Downloads/dog_vs_cat/test1/*")
test_image_list=[convert_image_to_data(i,HEIGHT,WIDTH) for i in test_files]
y_test_predict = model.predict(test_image_list)
df = pd.DataFrame(y_test)
df.to_csv("file_path.csv")